In [9]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

import warnings
import sklearn.exceptions
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

In [17]:
df = pd.read_csv('bank-additional-full.csv', delimiter=';')
df = df.replace(['unknown'], np.NaN)
df = df.dropna()
df['y'] = df['y'].map({'yes': 1, 'no': 0})

X = df.drop(['y'], axis=1)
y = df['y']
df

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
6,59,admin.,married,professional.course,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,1
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,0
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,0
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,1


# PERCENT POSITIVE

Finding percentage of positive label in the dataset

In [11]:
percent_pos = ((df['y'] == 1).sum() / len(df['y'])) * 100
percent_pos

12.65743899239045

In [12]:
# Transform X values into One Hot Encoding for categorical variables and Standardizing for numerical variables
# https://stackoverflow.com/questions/43798377/one-hot-encode-categorical-variables-and-scale-continuous-ones-simultaneouely

# Get categorical columns
cat = list(X.select_dtypes(['object']).columns)
# Get numerical columns
cont = list(X.select_dtypes(['int64']).columns)

# Scale numerical values
cont_transform = Pipeline(steps=[('scaler', StandardScaler())])
# Encode categorical values
cat_transform = Pipeline(steps=[('categories', OneHotEncoder(sparse=False, handle_unknown='ignore'))])
# Transform the dataset into the scaled and encoded version
preprocessor = ColumnTransformer(transformers=[('cont', cont_transform, cont),
                                               ('cat', cat_transform, cat)])
X = pd.DataFrame(preprocessor.fit_transform(X))
X

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,51
0,1.642253,0.005792,-0.559335,0.211887,-0.371616,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,-0.196452,-0.127944,-0.559335,0.211887,-0.371616,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.093870,-0.414520,-0.559335,0.211887,-0.371616,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.642253,0.181559,-0.559335,0.211887,-0.371616,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1.932575,-0.460373,-0.559335,0.211887,-0.371616,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30483,3.287410,0.284727,-0.559335,0.211887,-0.371616,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
30484,0.674513,0.471957,-0.559335,0.211887,-0.371616,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
30485,1.642253,-0.269321,-0.191702,0.211887,-0.371616,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
30486,0.480965,0.697398,-0.559335,0.211887,-0.371616,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


# LOGISTIC REGRESSION

In [13]:
def logisticRegression(X_train, X_test, y_train, y_test):
    train_metrics_log = pd.DataFrame(columns=['LG: Accuracy','LG: Precision','LG: AUC','LG: F1'])
    test_metrics_log = pd.DataFrame(columns=['LG: Accuracy','LG: Precision','LG: AUC','LG: F1'])
    
    pipe = Pipeline(steps=[('classifier', LogisticRegression())])

    # Setting Parameters to L1 and L2 regularized + unregularized model
    parameters = [{'classifier': [LogisticRegression(max_iter=5000, n_jobs=-1, class_weight='balanced')],
                   'classifier__solver': ['saga'],
                   'classifier__penalty': ['l1'],
                   'classifier__C': np.logspace(-8,4,13)},
                  {'classifier': [LogisticRegression(max_iter=5000, n_jobs=-1, class_weight='balanced')],
                   'classifier__solver': ['sag', 'saga'],
                   'classifier__penalty': ['none']},
                  {'classifier': [LogisticRegression(max_iter=5000, n_jobs=-1, class_weight='balanced')],
                   'classifier__solver': ['sag', 'saga'],
                   'classifier__penalty': ['l2'],
                   'classifier__C': np.logspace(-8,4,13)}]

    # Perform 5-fold cross-validation using grid search
    clf = GridSearchCV(pipe, parameters, cv=StratifiedKFold(n_splits=5), 
                       scoring=['accuracy', 'precision', 'roc_auc', 'f1'], refit=False, verbose=1)

    # Fitting training set on the cross validation
    hyperparams = clf.fit(X_train, y_train)
    # Storing best parameters for each metric model
    results = hyperparams.cv_results_['params']
    solution_log = pd.DataFrame(results)

    # ACCURACY MODEL
    solution_log['Accuracy'] = hyperparams.cv_results_['mean_test_accuracy']
    best_accuracy = results[np.argmin(hyperparams.cv_results_['rank_test_accuracy'])]
    # creating new model with optimal hyperparameters
    if 'classifier__C' in best_accuracy:
        accuracy_model = LogisticRegression(penalty = best_accuracy['classifier__penalty'],
                                            C = best_accuracy['classifier__C'],
                                            solver = best_accuracy['classifier__solver'],
                                            max_iter = 5000,
                                            n_jobs = -1,
                                            class_weight='balanced')
    else:
        accuracy_model = LogisticRegression(penalty = best_accuracy['classifier__penalty'],
                                            solver = best_accuracy['classifier__solver'],
                                            max_iter = 5000,
                                            n_jobs = -1,
                                            class_weight='balanced')
    # Training on the new model
    accuracy_model.fit(X_train, y_train)
    y_acc_train = accuracy_model.predict(X_train)
    acc_train_score = accuracy_score(y_train, y_acc_train)
    
    # Testing and scoring the model
    y_acc_test = accuracy_model.predict(X_test)
    acc_test_score = accuracy_score(y_test, y_acc_test)

    # PRECISION MODEL
    solution_log['Precision'] = hyperparams.cv_results_['mean_test_precision']
    best_precision = results[np.argmin(hyperparams.cv_results_['rank_test_precision'])]
    
    # creating new model with optimal hyperparameters
    if 'classifier__C' in best_precision:
        precision_model = LogisticRegression(penalty = best_precision['classifier__penalty'],
                                             C = best_precision['classifier__C'],
                                             solver = best_precision['classifier__solver'],
                                             max_iter = 5000,
                                             n_jobs = -1,
                                             class_weight='balanced')
    else:
        precision_model = LogisticRegression(penalty = best_precision['classifier__penalty'],
                                             solver = best_precision['classifier__solver'],
                                             max_iter = 5000,
                                             n_jobs = -1,
                                             class_weight='balanced')
    # Training on the new model
    precision_model.fit(X_train, y_train)
    y_prec_train = precision_model.predict(X_train)
    prec_train_score = precision_score(y_train, y_prec_train)

    # Testing and scoring the model
    y_prec_test = precision_model.predict(X_test)
    prec_test_score = precision_score(y_test, y_prec_test)

    # ROC AUC MODEL
    solution_log['ROC AUC'] = hyperparams.cv_results_['mean_test_roc_auc']
    best_roc_auc = results[np.argmin(hyperparams.cv_results_['rank_test_roc_auc'])]
    
    # creating new model with optimal hyperparameters
    if 'classifier__C' in best_roc_auc:
        roc_model = LogisticRegression(penalty = best_roc_auc['classifier__penalty'],
                                       C = best_roc_auc['classifier__C'],
                                       solver = best_roc_auc['classifier__solver'],
                                       max_iter = 5000,
                                       n_jobs = -1,
                                       class_weight='balanced')
    else:
        roc_model = LogisticRegression(penalty = best_roc_auc['classifier__penalty'],
                                       solver = best_roc_auc['classifier__solver'],
                                       max_iter = 5000,
                                       n_jobs = -1,
                                       class_weight='balanced')
    # Training on the new model
    roc_model.fit(X_train, y_train)
    y_roc_train = roc_model.predict(X_train)
    roc_train_score = roc_auc_score(y_train, y_roc_train)
    
    # Testing and scoring the model
    y_roc_test = roc_model.predict(X_test)
    roc_test_score = roc_auc_score(y_test, y_roc_test)

    # F1 MODEL
    solution_log['F1'] = hyperparams.cv_results_['mean_test_f1']
    best_f1 = results[np.argmin(hyperparams.cv_results_['rank_test_f1'])]
    
    
    # creating new model with optimal hyperparameters
    if 'classifier__C' in best_f1:
        f1_model = LogisticRegression(penalty = best_f1['classifier__penalty'],
                                      C = best_f1['classifier__C'],
                                      solver = best_f1['classifier__solver'],
                                      max_iter = 5000,
                                      n_jobs = -1,
                                      class_weight='balanced')
    else:
        f1_model = LogisticRegression(penalty = best_f1['classifier__penalty'],
                                      solver = best_f1['classifier__solver'],
                                      max_iter = 5000,
                                      n_jobs = -1,
                                      class_weight='balanced')
    
    # Training on the new model
    f1_model.fit(X_train, y_train)
    y_f1_train = f1_model.predict(X_train)
    f1_train_score = f1_score(y_train, y_f1_train)
    
    # Testing and scoring the model
    y_f1_test = f1_model.predict(X_test)
    f1_test_score = f1_score(y_test, y_f1_test)

    train_metrics_log = train_metrics_log.append({'LG: Accuracy': acc_train_score, 'LG: Precision': prec_train_score, 
                                          'LG: AUC': roc_train_score, 'LG: F1': f1_train_score}, ignore_index=True)

    test_metrics_log = test_metrics_log.append({'LG: Accuracy': acc_test_score, 'LG: Precision': prec_test_score, 
                                          'LG: AUC': roc_test_score, 'LG: F1': f1_test_score}, ignore_index=True)
    
    return train_metrics_log, test_metrics_log, solution_log

# RANDOM FOREST CLASSIFIER

In [14]:
 def randomForest(X_train, X_test, y_train, y_test):
    train_metrics_rf = pd.DataFrame(columns=['RF: Accuracy','RF: Precision','RF: AUC','RF: F1'])
    test_metrics_rf = pd.DataFrame(columns=['RF: Accuracy','RF: Precision','RF: AUC','RF: F1'])

    randomForest = RandomForestClassifier()

    # Setting parameters according to CNM06
    param_grid = {
        'n_estimators': [1024],
        'criterion': ['gini', 'entropy'],
        'max_features': [1,2,4,6,8,12,16],
        'n_jobs': [-1],
        'class_weight': ['balanced']}

    # Perform 5-fold cross-validation using grid search
    clf = GridSearchCV(estimator=randomForest, param_grid=param_grid, cv=StratifiedKFold(n_splits=5), 
                       scoring=['accuracy', 'precision', 'roc_auc', 'f1'], refit=False, verbose=1)

    # Fitting training set on the cross validation
    hyperparams = clf.fit(X_train, y_train)
    # Storing best parameters for each metric model
    results = hyperparams.cv_results_['params']
    solution_rf = pd.DataFrame(results)

    # ACCURACY MODEL
    solution_rf['Accuracy'] = hyperparams.cv_results_['mean_test_accuracy']
    best_accuracy = results[np.argmin(hyperparams.cv_results_['rank_test_accuracy'])]
    # creating new model with optimal hyperparameters
    accuracy_model = RandomForestClassifier(n_estimators = best_accuracy['n_estimators'],
                                            criterion = best_accuracy['criterion'],
                                            max_features = best_accuracy['max_features'],
                                            n_jobs = -1,
                                            class_weight='balanced')
    # Training on the new model
    accuracy_model.fit(X_train, y_train)
    y_acc_train = accuracy_model.predict(X_train)
    acc_train_score = accuracy_score(y_train, y_acc_train)

    # Testing and scoring the model
    y_acc_test = accuracy_model.predict(X_test)
    acc_test_score = accuracy_score(y_test, y_acc_test)

    # PRECISION MODEL
    solution_rf['Precision'] = hyperparams.cv_results_['mean_test_precision']
    best_precision = results[np.argmin(hyperparams.cv_results_['rank_test_precision'])]
    # creating new model with optimal hyperparameters
    precision_model = RandomForestClassifier(n_estimators = best_precision['n_estimators'],
                                             criterion = best_precision['criterion'],
                                             max_features = best_precision['max_features'],
                                             n_jobs = -1,
                                             class_weight='balanced')
    # Training on the new model
    precision_model.fit(X_train, y_train)
    y_prec_train = precision_model.predict(X_train)
    prec_train_score = precision_score(y_train, y_prec_train)

    # Testing and scoring the model
    y_prec_test = precision_model.predict(X_test)
    prec_test_score = precision_score(y_test, y_prec_test)

    # ROC AUC MODEL
    solution_rf['ROC AUC'] = hyperparams.cv_results_['mean_test_roc_auc']
    best_roc_auc = results[np.argmin(hyperparams.cv_results_['rank_test_roc_auc'])]
    # creating new model with optimal hyperparameters
    roc_model = RandomForestClassifier(n_estimators = best_roc_auc['n_estimators'],
                                       criterion = best_roc_auc['criterion'],
                                       max_features = best_roc_auc['max_features'],
                                       n_jobs = -1,
                                       class_weight='balanced')
    # Training on the new model
    roc_model.fit(X_train, y_train)
    y_roc_train = roc_model.predict(X_train)
    roc_train_score = roc_auc_score(y_train, y_roc_train)

    # Testing and scoring the model
    y_roc_test = roc_model.predict(X_test)
    roc_test_score = roc_auc_score(y_test, y_roc_test)

    # F1 MODEL
    solution_rf['F1'] = hyperparams.cv_results_['mean_test_f1']
    best_f1 = results[np.argmin(hyperparams.cv_results_['rank_test_f1'])]
    # creating new model with optimal hyperparameters
    f1_model = RandomForestClassifier(n_estimators = best_f1['n_estimators'],
                                      criterion = best_f1['criterion'],
                                      max_features = best_f1['max_features'],
                                      n_jobs = -1,
                                      class_weight='balanced')
    # Training on the new model
    f1_model.fit(X_train, y_train)
    y_f1_train = f1_model.predict(X_train)
    f1_train_score = f1_score(y_train, y_f1_train)

    # Testing and scoring the model
    y_f1_test = f1_model.predict(X_test)
    f1_test_score = f1_score(y_test, y_f1_test)

    train_metrics_rf = train_metrics_rf.append({'RF: Accuracy': acc_train_score, 'RF: Precision': prec_train_score,
                                                'RF: AUC': roc_train_score, 'RF: F1': f1_train_score}, ignore_index=True)

    test_metrics_rf = test_metrics_rf.append({'RF: Accuracy': acc_test_score, 'RF: Precision': prec_test_score,
                                              'RF: AUC': roc_test_score, 'RF: F1': f1_test_score}, ignore_index=True)
    
    return train_metrics_rf, test_metrics_rf, solution_rf

# DECISION TREE CLASSIFIER

In [15]:
def decisionTrees(X_train, X_test, y_train, y_test):
    train_metrics_dt = pd.DataFrame(columns=['DT: Accuracy','DT: Precision','DT: AUC','DT: F1'])
    test_metrics_dt = pd.DataFrame(columns=['DT: Accuracy','DT: Precision','DT: AUC','DT: F1'])

    pipe = Pipeline(steps=[('classifier', DecisionTreeClassifier())])

    # Setting parameters according to CNM06 + passing a list of min_samples_leaf
    parameters = [{'classifier': [DecisionTreeClassifier(class_weight='balanced')],
                   'classifier__criterion': ['gini', 'entropy'],
                   'classifier__splitter': ['best'],
                   'classifier__min_samples_leaf': [1,2,4,6,8,10,12,14,16,18]}]

    # Perform 5-fold cross-validation using grid search
    clf = GridSearchCV(estimator=pipe, param_grid=parameters, cv=StratifiedKFold(n_splits=5), 
                       scoring=['accuracy', 'precision', 'roc_auc', 'f1'], refit=False, verbose=1)
    
    # Fitting training set on the cross validation
    hyperparams = clf.fit(X_train, y_train)
    # Storing best parameters for each metric model
    results = hyperparams.cv_results_['params']
    solution_dt = pd.DataFrame(results)

    # ACCURACY
    solution_dt['Accuracy'] = hyperparams.cv_results_['mean_test_accuracy']
    best_accuracy = results[np.argmin(hyperparams.cv_results_['rank_test_accuracy'])]
    # creating new model with optimal hyperparameters
    accuracy_model = DecisionTreeClassifier(criterion = best_accuracy['classifier__criterion'],
                                            splitter = best_accuracy['classifier__splitter'],
                                            min_samples_leaf = best_accuracy['classifier__min_samples_leaf'],
                                            class_weight = 'balanced')

    # Training on the new model
    accuracy_model.fit(X_train, y_train)
    y_acc_train = accuracy_model.predict(X_train)
    acc_train_score = accuracy_score(y_train, y_acc_train)

    # Testing and scoring the model
    y_acc_test = accuracy_model.predict(X_test)
    acc_test_score = accuracy_score(y_test, y_acc_test)

    # PRECISION
    solution_dt['Precision'] = hyperparams.cv_results_['mean_test_precision']
    best_precision = results[np.argmin(hyperparams.cv_results_['rank_test_precision'])]
    # creating new model with optimal hyperparameters
    precision_model = DecisionTreeClassifier(criterion = best_precision['classifier__criterion'],
                                             splitter = best_precision['classifier__splitter'],
                                             min_samples_leaf = best_precision['classifier__min_samples_leaf'],
                                             class_weight = 'balanced')
    # Training on the new model
    precision_model.fit(X_train, y_train)
    y_prec_train = precision_model.predict(X_train)
    prec_train_score = precision_score(y_train, y_prec_train)

    # Testing and scoring the model
    y_prec_test = precision_model.predict(X_test)
    prec_test_score = precision_score(y_test, y_prec_test)

    # ROC AUC
    solution_dt['ROC AUC'] = hyperparams.cv_results_['mean_test_roc_auc']
    best_roc_auc = results[np.argmin(hyperparams.cv_results_['rank_test_roc_auc'])]
    # creating new model with optimal hyperparameters
    roc_model = DecisionTreeClassifier(criterion = best_roc_auc['classifier__criterion'],
                                       splitter = best_roc_auc['classifier__splitter'],
                                       min_samples_leaf = best_roc_auc['classifier__min_samples_leaf'],
                                       class_weight = 'balanced')
    # Training on the new model
    roc_model.fit(X_train, y_train)
    y_roc_train = roc_model.predict(X_train)
    roc_train_score = roc_auc_score(y_train, y_roc_train)

    # Testing and scoring the model
    y_roc_test = roc_model.predict(X_test)
    roc_test_score = roc_auc_score(y_test, y_roc_test)

    # F1
    solution_dt['F1'] = hyperparams.cv_results_['mean_test_f1']
    best_f1 = results[np.argmin(hyperparams.cv_results_['rank_test_f1'])]
    # creating new model with optimal hyperparameters
    f1_model = DecisionTreeClassifier(criterion = best_f1['classifier__criterion'],
                                      splitter = best_f1['classifier__splitter'],
                                      min_samples_leaf = best_f1['classifier__min_samples_leaf'],
                                      class_weight = 'balanced')
    # Training on the new model
    f1_model.fit(X_train, y_train)
    y_f1_train = f1_model.predict(X_train)
    f1_train_score = f1_score(y_train, y_f1_train)

    # Testing and scoring the model
    y_f1_test = f1_model.predict(X_test)
    f1_test_score = f1_score(y_test, y_f1_test)

    train_metrics_dt = train_metrics_dt.append({'DT: Accuracy': acc_train_score, 'DT: Precision': prec_train_score,
                                                'DT: AUC': roc_train_score, 'DT: F1': f1_train_score}, ignore_index=True)

    test_metrics_dt = test_metrics_dt.append({'DT: Accuracy': acc_test_score, 'DT: Precision': prec_test_score,
                                              'DT: AUC': roc_test_score, 'DT: F1': f1_test_score}, ignore_index=True)
    
    return train_metrics_dt, test_metrics_dt, solution_dt

# RUNNING DATASET ON ALL THREE ALGORITHMS

In [16]:
trials = 5
train_metrics = pd.DataFrame()
test_metrics = pd.DataFrame()
solution_metrics = pd.DataFrame()

# Running the trial five times
for i in range(trials):
    # Splitting data into train size = 5000
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=5000, shuffle=True)
    
    train_log, test_log, solution_log = logisticRegression(X_train, X_test, y_train, y_test)
    train_rf, test_rf, solution_rf = randomForest(X_train, X_test, y_train, y_test)
    train_dt, test_dt, solution_dt = decisionTrees(X_train, X_test, y_train, y_test)

    train_metrics = train_metrics.append(pd.concat([train_log, train_rf, train_dt], axis=1))
    test_metrics = test_metrics.append(pd.concat([test_log, test_rf, test_dt], axis=1))
    solution_metrics = solution_metrics.append(pd.concat([solution_log, solution_rf, solution_dt], axis=1))

# storing data into CSV file
train_metrics.to_csv('bank_train.csv')
test_metrics.to_csv('bank_test.csv')
solution_metrics.to_csv('bank_solution.csv')

Fitting 5 folds for each of 41 candidates, totalling 205 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 205 out of 205 | elapsed: 49.3min finished


Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed: 19.7min finished


Fitting 5 folds for each of 22 candidates, totalling 110 fits


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 205 out of 205 | elapsed: 12.7min finished


Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:  6.0min finished


Fitting 5 folds for each of 22 candidates, totalling 110 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 110 out of 110 | elapsed:    3.2s finished
